In [ ]:
# mamba install pyproj

In [1]:
import rasterio
import numpy as np
from pyproj import CRS, Transformer
from rasterio.warp import calculate_default_transform, Resampling,reproject
from pyproj.database import query_utm_crs_info
from pyproj.aoi import AreaOfInterest

In [2]:
file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/input/LC_HAZ_DUMMY_Australia.tif"
out_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_HAZ_DUMMY_AUS_UTM.tif"

In [4]:
def is_utm_epsg(epsg_code):
    try:
        crs = CRS.from_epsg(epsg_code)
        return crs.coordinate_operation.method_name == "Transverse Mercator"
    except Exception as e:
        return False


def utm_finder(src_crs,bbox=[]):
    try:
        """
        Find UTM epsg
        raster: input raster path
        Returns:
        UTM EPSG code of the input raster
        """
        # with rasterio.open(raster_file_path) as dataset:  
            # src_epsg=dataset.crs.to_epsg()
            # bbox  = dataset.bounds
        bbox_wgs84 = rasterio.warp.transform_bounds(src_crs,'EPSG:4326', bbox[0],bbox[1],bbox[2],bbox[3])
        utm_crs_list = query_utm_crs_info(     
            datum_name='WGS 84',
            area_of_interest= AreaOfInterest(
            west_lon_degree=bbox_wgs84[0],
            south_lat_degree=bbox_wgs84[1],
            east_lon_degree=bbox_wgs84[2],
            north_lat_degree=bbox_wgs84[3],),) 

        # utm_crs = '{}:{}'.format(utm_crs_list[0].auth_name,utm_crs_list[0].code)
        utm_epsg = utm_crs_list[0].code
        print(utm_epsg,"utm_epsg")
        return True,utm_epsg
    except Exception as e:
        return False, str(e)

In [5]:
ds = rasterio.open(file_path)
#check raster crs and reproject if necessary
ds_crs_epsg=ds.crs.to_epsg()
is_UTM_epsg=is_utm_epsg(ds_crs_epsg)

print(ds_crs_epsg)
print(is_UTM_epsg)

if not is_UTM_epsg:
    bbox  = ds.bounds
    success,dst_utm_epsg=utm_finder(ds_crs_epsg,bbox)
    if not success:
        error=str(dst_utm_epsg)
    src_crs=CRS.from_epsg(ds_crs_epsg) 
    dst_crs = CRS.from_epsg(dst_utm_epsg) 
    transformer=Transformer.from_crs(src_crs,dst_crs)
    dst_transform, width, height = calculate_default_transform(
        src_crs.to_string(), dst_crs.to_string(), ds.width, ds.height, *ds.bounds
    )
    kwargs = ds.meta.copy()
    kwargs.update({
                'crs': dst_crs.to_string(),
                'transform': dst_transform,
                'width': width,
                'height': height})
    with rasterio.open(out_path, 'w', **kwargs) as dst:
        for i in range(1, ds.count + 1):
            reproject(
                source=rasterio.band(ds, i),
                destination=rasterio.band(dst, i),
                src_transform=ds.transform,
                src_crs=src_crs.to_string(),
                dst_transform=dst_transform,
                dst_crs=dst_crs.to_string(),
                resampling=Resampling.nearest)
    ds.close()
    dst.close()
    # ds = rasterio.open(instance.file.path)

3857
False
32749 utm_epsg


END

In [ ]:
# file_path='/Users/rabina/Projects/sdss/sdss_conda/data/FL_DE_50_A0.tif'
# out_path='/Users/rabina/Projects/sdss/sdss_conda/data/FL_DE_50_A0.tif'
file_path="/Users/rabina/Projects/sdss/sdss_conda/data/ear_raster/output/lc_romania_reclassified.tif"
out_path="/Users/rabina/Projects/sdss/sdss_conda/data/ear_raster/output/lc_romania_reclassified.tif"
ds = rasterio.open(file_path)
src_crs=ds.crs
print(src_crs.to_epsg())
if src_crs!=4326:
    print("not equal")
    # crs_4326 = CRS.from_epsg(4326) 
    crs_4326 = CRS(
        proj='longlat',
        datum='WGS84',
        unit='metre'
    )

    transformer=Transformer.from_crs(src_crs,crs_4326)
    dst_transform, width, height = calculate_default_transform(
        src_crs, crs_4326, ds.width, ds.height, *ds.bounds
    )
    # print(src_crs)
    kwargs = ds.meta.copy()
    kwargs.update({
                'crs': crs_4326,
                'transform': dst_transform,
                'width': width,
                'height': height})

    with rasterio.open(out_path, 'w', **kwargs) as dst:
        for i in range(1, ds.count + 1):
            reproject(
                source=rasterio.band(ds, i),
                destination=rasterio.band(dst, i),
                src_transform=ds.transform,
                src_crs=ds.crs,
                dst_transform=dst_transform,
                dst_crs=crs_4326,
                resampling=Resampling.nearest)

In [ ]:
print(crs_4326.to_wkt())

END

In [ ]:
crs_4326 = CRS.from_epsg(4326) 
transformer=Transformer.from_crs(src_crs,crs_4326)
dst_transform, width, height = calculate_default_transform(
    src_crs, crs_4326, ds.width, ds.height, *ds.bounds
)
print(width,height)

In [ ]:
crs_4326_data = ds.read(
    out_shape=(ds.count, height, width),
    resampling=Resampling.nearest,
)